In [10]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

**keys**: homogeneous Neumann bvp, hypersingular operator $D$

Neumann Laplace Indirect Method
======================================================

Consider the Neumann boundary value problem 

$$ \left\{ \begin{array}{rlc l} \Delta u &=& 0, \quad &\Omega \subset \mathbb R^3\,,\\ \gamma_1 u&=& j, \quad &\Gamma = \partial \Omega\,.\end{array} \right. $$ 

The solution $u\in H^1(\Omega)$ of the above bvp can be written in the following form (representation forumula) 

$$ u(x) =  \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{ \langle n(y), x-y \rangle }{\| x-y\|^3} } \, m(y)\, \mathrm{d}s_y}_{\displaystyle{ \mathrm{DL}(m) }}\,.$$ 


**NGSolve Solution** 

Define the geometry $\Omega \subset \mathbb R^3$ and create a mesh:

In [11]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.3)).Curve(2)

Define the finite element space for $H^{\frac12}(\Gamma)$ for given geometry :  

In [12]:
fesH1 = H1(mesh, order=2, definedon=mesh.Boundaries(".*"))
uH1,vH1 = fesH1.TnT()
print ("ndof H1 = ", fesH1.ndof)

ndof H1 =  630


**Boundary Integral Equation** 

Let's carefully apply the Neumann trace $\gamma_1$ to the representation formula and solve the resulting boundary integral equation for $m \in H^{\frac12}(\Gamma)$ by discretisation of the following variational formulation: 

$$\begin{array}{r c ccc} \forall \, v\in H^{\frac12}(\Gamma): && \displaystyle \int\limits_\Gamma v \cdot \gamma_1 \left(\mathrm{DL}(m) \right) \, \mathrm d s  &=&  \displaystyle \int\limits_\Gamma j \cdot  v \,\mathrm d s \\[2ex] & \Rightarrow & \mathrm{D}\,\mathrm{m} &=& \mathrm{M} \, \mathrm j \,. \end{array} $$ 


The discretisation of the variational formulation leads to a system of linear equations, ie 

$$ \left( \mathrm{D} + \mathrm{S}\right) \, \mathrm{m}= \mathrm{M}\, \mathrm j\,, $$ 

where $\mathrm{D}$ is the hypersingular operator and the stabilisation $(\mathrm D + \mathrm{S})$ is regular and symmetric and $\mathrm j$ is the given Neumann data.

In [13]:
u_exa = 1/ sqrt( (x-1)**2 + (y-1)**2 + (z-1)**2 )
gradu_exa = CF( (u_exa.Diff(x), u_exa.Diff(y), u_exa.Diff(z)) )

n = specialcf.normal(3)
j = gradu_exa*n
rhs = LinearForm(j*vH1.Trace()*ds(bonus_intorder=3)).Assemble()

In [14]:
vH1m1 = LinearForm(vH1*1*ds(bonus_intorder=3)).Assemble()
S = (BaseMatrix(Matrix(vH1m1.vec.Reshape(1))))@(BaseMatrix(Matrix(vH1m1.vec.Reshape(fesH1.ndof))))

The assembly of the hypersingular operator $D$ is declarative and close to their actual implementations (explicit formula are given below).

In [32]:
D = LaplaceSL(-curl(uH1)*ds) * curl(vH1)*ds # ZU KLÄREN VORZEICHEN irgendwo geht ein VORZEICHEN SCHIEF

In [33]:
m = GridFunction(fesH1)
pre = BilinearForm(uH1*vH1*ds).Assemble().mat.Inverse(freedofs=fesH1.FreeDofs()) 
with TaskManager(): 
    CG(mat = D.mat+S, pre=pre, rhs = rhs.vec, sol=m.vec, tol=1e-8, maxsteps=200, initialize=False, printrates=True)

CG iteration 1, residual = 1.0926249988826613     
CG iteration 2, residual = 0.5729010293377176     
CG iteration 3, residual = 0.34312530188590223     
CG iteration 4, residual = 0.2303924026796566     
CG iteration 5, residual = 0.17840078203740353     
CG iteration 6, residual = 0.11050149092809973     
CG iteration 7, residual = 0.07041199040456789     
CG iteration 8, residual = 0.0424621243976524     
CG iteration 9, residual = 0.026992974308580005     
CG iteration 10, residual = 0.016201774986785808     
CG iteration 11, residual = 0.01864346924208354     
CG iteration 12, residual = 0.02056086215637912     
CG iteration 13, residual = 0.006946861001430852     
CG iteration 14, residual = 0.003687515147986319     
CG iteration 15, residual = 0.001856658345736144     
CG iteration 16, residual = 0.0009253109947845533     
CG iteration 17, residual = 0.0004411780375487626     
CG iteration 18, residual = 0.00020689857099380433     
CG iteration 19, residual = 9.382945877775458e-

In [34]:
Draw (m, mesh, draw_vol=False, order=3);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

**Evaluation of the solution**

In [43]:
screen = WorkPlane(Axes( (0,0,0), Z, X)).RectangleC(0.5,0.5).Face()
vismesh = screen.GenerateMesh(maxh=0.05)
#Draw (vismesh);
LDL = LaplaceDL( uH1*ds)
u_num = LDL(m)
u_num.BuildLocalExpansion(vismesh.Boundaries(".*")) #MLFMM evaluation
Draw(u_num+0.575, vismesh) # Übereinstimmung bis auf Konstante
Draw(u_exa, vismesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

**Details** 

For any $u_j, v_i \in H^{\frac12}(\Gamma)$, hypersingular operator is implemented as follows (reference below)

$$ D_{ij} = \big\langle v_i, \gamma_1 \mathrm{DL}(u_j) \big\rangle = \frac{1}{4\pi} \int\limits_\Gamma\int\limits_\Gamma \frac{ \langle \mathrm{\boldsymbol {curl}}_\Gamma  \,u_j(x), \mathrm{\boldsymbol{curl}}_\Gamma\, v_i(y) \rangle}{\|x-y\|} \, \mathrm{d} s_{y} \, \mathrm{d} s_x $$ 

This explicit represenation of $D$ is the singular layer potential operator with vector valued surface curl operator applied on trial and test functions. You may assemble $D$ it by

```
D = LaplaceSL(curl(u)*ds)*curl(v)*ds
```

**References**

For details on the explicit representation of the hypersingular operator have a look into [Numerische Näherungsverfahren für elliptische Randwertprobleme](https://link.springer.com/book/10.1007/978-3-322-80054-1), p.127, p.259 (1st edition).